[View in Colaboratory](https://colab.research.google.com/github/nozakijpn/kenkyu/blob/master/DNCNN_for_audio.ipynb)

# DeNoising Convolutional Neural Network for Cifar dataset

In [5]:

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive


'\n!apt-get install -y -qq software-properties-common python-software-properties module-init-tools\n!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null\n!apt-get update -qq 2>&1 > /dev/null\n!apt-get -y install -qq google-drive-ocamlfuse fuse\n\nfrom google.colab import auth\nauth.authenticate_user()\nfrom oauth2client.client import GoogleCredentials\ncreds = GoogleCredentials.get_application_default()\n\nimport getpass\n!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL\nvcode = getpass.getpass()\n!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}\n\n!mkdir -p drive\n!google-drive-ocamlfuse drive\n!ls drive\n'

In [116]:
%cd /content/drive/study/news

/content/drive/study/news


In [117]:
!pip install librosa

In [118]:
import glob
filelist = glob.glob("NHK0826*.wav")
print(filelist)

['NHK0826_0072.wav', 'NHK0826_0079.wav', 'NHK0826_0069.wav', 'NHK0826_0078.wav', 'NHK0826_0085.wav', 'NHK0826_0070.wav', 'NHK0826_0083.wav', 'NHK0826_0074.wav', 'NHK0826_0073.wav', 'NHK0826_0041.wav', 'NHK0826_0087.wav', 'NHK0826_0088.wav', 'NHK0826_0090.wav', 'NHK0826_0059.wav', 'NHK0826_0096.wav', 'NHK0826_0099.wav', 'NHK0826_0089.wav', 'NHK0826_0097.wav', 'NHK0826_0095.wav', 'NHK0826_0091.wav', 'NHK0826_0093.wav', 'NHK0826_0098.wav', 'NHK0826_0060.wav', 'NHK0826_0100.wav', 'NHK0826_0092.wav', 'NHK0826_0101.wav', 'NHK0826_0064.wav', 'NHK0826_0105.wav', 'NHK0826_0111.wav', 'NHK0826_0104.wav', 'NHK0826_0110.wav', 'NHK0826_0071.wav', 'NHK0826_0112.wav', 'NHK0826_0109.wav', 'NHK0826_0103.wav', 'NHK0826_0102.wav', 'NHK0826_0106.wav', 'NHK0826_0107.wav', 'NHK0826_0108.wav', 'NHK0826_0113.wav', 'NHK0826_0075.wav', 'NHK0826_0114.wav', 'NHK0826_0116.wav', 'NHK0826_0117.wav', 'NHK0826_0077.wav', 'NHK0826_0115.wav', 'NHK0826_0119.wav', 'NHK0826_0080.wav', 'NHK0826_0081.wav', 'NHK0826_0082.wav',

globでデータのリストをとってきて、スペクトログラムの配列を作成してみる

In [0]:
import wave
import numpy as np
import cmath as cm
import matplotlib.pyplot as plt
import scipy
import librosa
import librosa.display
# 波形データ読み込み
path = "NHK0826_0008.wav"
#path = "/home/nozaki/TX0528.wav"
#sample_rate, sound = scipy.io.wavfile.read(librosa.util.example_audio_file())
for i,item in enumerate(filelist):
  sound,sample_rate = librosa.load(item,sr=44100)
  """"
  
  print("sample rate {}".format(sample_rate))
  second = sound.shape[0]/sample_rate
  minute = int(second/60)
  print("{0}分{1}秒".format(minute,int(second%60)))
  """
  window_size = 0.2
  window_stride = 0.1
  n_fft = int(sample_rate * window_size)
  win_length = n_fft
  hop_length = int(sample_rate * window_stride)
  # 短時間フーリエ変換
  #D = librosa.stft(sound, n_fft=n_fft, hop_length=hop_length,win_length=win_length)
  D = librosa.stft(sound)
  if(i > 0):
    D_sub = np.hstack((D,D_sub))
    #print(D.shape,D_sub.shape)
  else:
    D_sub = D

In [120]:
print(D.shape)
print(np.min(D_sub),np.max(D_sub))
print(D_sub.shape)

(1025, 3324)
((-72.46452+28.665735j), (70.16213-13.621045j))
(1025, 140703)


In [0]:
import numpy as np
def min_max(x, axis=None):
    min = x.min(axis=axis, keepdims=True)
    max = x.max(axis=axis, keepdims=True)
    result = (x-min)/(max-min)
    return result

In [122]:
D1 = min_max(D)
print(D1.shape)

(1025, 3324)


In [123]:
print(D_sub.shape)

(1025, 140703)


In [124]:
#D_sub1 = min_max(D_sub)
D_sub1 = D_sub
D_sub1 = D_sub1.transpose(1,0)
print(D_sub1.shape)
cut = int(9*D_sub1.shape[0]/10)
x_train , x_test = np.split(D_sub1, [cut],axis=0)
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1]/5,5,1))
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1]/5,5,1))
print(x_train.shape)
print(x_test.shape)

(140703, 1025)
(126632, 205, 5, 1)
(14071, 205, 5, 1)


In [125]:
!pip install keras
!pip install torch
!pip install torchvision

## Backend and Import

In [126]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
kerasBKED = os.environ["KERAS_BACKEND"] 
print(kerasBKED)

tensorflow


In [0]:
import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import os
import pickle
import numpy as np

In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## Load Cifar10 dataset

In [0]:
batch_size = 100
epochs = 10
saveDir = "./"
if not os.path.isdir(saveDir):
    os.makedirs(saveDir)

In [0]:
x_train_line = np.copy(x_train)
x_test_line = np.copy(x_test)
#x_train_line = drawLines(x_train_line)
#x_test_line = drawLines(x_test_line)

In [131]:
print(x_train_line.shape)

(126632, 205, 5, 1)


## normalize data

In [132]:
# normalize data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255

# normalize data
x_train_line = x_train_line.astype('float32')
x_test_line = x_test_line.astype('float32')
#x_train_line /= 255
#x_test_line /= 255

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [133]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

('x_train shape:', (126632, 205, 5, 1))
(126632, 'train samples')
(14071, 'test samples')


## show noisy images

## DNCNN

In [0]:
input_img = Input(shape=(x_train.shape[1], 5, 1))
x = Conv2D(64, (3,3), padding='same')(input_img)
x = Activation('relu')(x)

for i in range(15):
    x = Conv2D(64, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

x = Conv2D(1, (3,3), padding='same')(x)
output_img = Activation('tanh')(x)

In [0]:
model = Model(input_img, output_img)
model.compile(optimizer='adam', loss='mean_squared_error')


## Train CNN

In [0]:
# load pretrained weights
#model.load_weights(saveDir + "AutoEncoder_Cifar10_dncnn_weights.02-0.00-0.00.hdf5")

In [0]:
es_cb = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')
chkpt = saveDir + 'AutoEncoder_Cifar10_dncnn_weights.{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5'
cp_cb = ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [0]:
history = model.fit(x_train_line, x_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test_line, x_test),
                    callbacks=[es_cb, cp_cb],
                    shuffle=True)

In [0]:
#model.load_weights('./AutoEncoder_Cifar10_dncnn_weights.01-0.00-0.00.hdf5')

## Evaluate with test dataset

In [89]:
score = model.evaluate(x_test_line, x_test, verbose=1)
print(score)

6400/6400 [==============================] - 46s 7ms/step
0.7707107844948768


In [0]:
pred = model.predict(x_test_line)

In [91]:
print(x_test_line.shape)
print(pred.shape)

(6400, 4411, 1, 1)
(6400, 4411, 1, 1)


## Visualize original image, noisy image and denoised image

音声の復元メソッド

In [0]:
#y = librosa.istft(D,hop_length=hop_length,win_length=win_length)
#y = librosa.istft(D)
#librosa.output.write_wav("test1.wav",y,44100)